In [1]:
"""
Run the preprocessing pipeline with 
the fusion decomposition that will 
be packaged up into preprocessing version 2
"""



'\nRun the preprocessing pipeline with \nthe fusion decomposition that will \nbe packaged up into preprocessing version 2\n'

In [30]:
tu = reload(tu)

In [2]:
from os import sys
sys.path.append("/meshAfterParty/")

In [235]:
import skeleton_utils as sk
import soma_extraction_utils as sm
import trimesh_utils as tu
import trimesh
import numpy_utils as nu
import numpy as np
from importlib import reload
import networkx as nx
import time
import compartment_utils as cu
import networkx_utils as xu
import matplotlib_utils as mu

#importing at the bottom so don't get any conflicts
import itertools
from tqdm_utils import tqdm

#for meshparty preprocessing
import meshparty_skeletonize as m_sk
import general_utils as gu
import compartment_utils as cu
from meshparty import trimesh_io
from copy import deepcopy

from neuron_utils import *


# Loading the Neuron

In [3]:
import trimesh_utils as tu
curent_neuron = tu.load_mesh_no_processing("/notebooks/test_neurons/Segmentation_2/meshparty/864691135548568516_single_soma_inhib_axon_cloud.off")

In [4]:
segment_id = 864691135548568516
description = "single_soma_inhib_axon_cloud"

In [5]:
#predefined arguments for the Neuron constructor

decomposition_type="meshafterparty"
mesh_correspondence="meshparty" #meshafterparty_adaptive
distance_by_mesh_center=True #how the distance is calculated for mesh correspondence
meshparty_segment_size = 100
meshparty_n_surface_downsampling = 2
meshparty_adaptive_correspondence_after_creation=False
suppress_preprocessing_print=True
computed_attribute_dict=None
somas = None
branch_skeleton_data=None
combine_close_skeleton_nodes = True
combine_close_skeleton_nodes_threshold=700
ignore_warnings=True
suppress_output=False
calculate_spines=True
widths_to_calculate=["no_spine_median_mesh_center",
                "no_spine_mean_mesh_center"]
fill_hole_size=2000

In [16]:
#arguments for the preprocess neuron
mesh = curent_neuron
segment_id=segment_id
description=description

sig_th_initial_split=15 #for significant splitting meshes in the intial mesh split
limb_threshold = 2000 #the mesh faces threshold for a mesh to be qualified as a limb (otherwise too small)
filter_end_node_length=4001 #used in cleaning the skeleton during skeletonizations
return_no_somas = False

decomposition_type=decomposition_type
mesh_correspondence=mesh_correspondence
distance_by_mesh_center=distance_by_mesh_center
meshparty_segment_size =meshparty_segment_size
meshparty_n_surface_downsampling = meshparty_n_surface_downsampling
somas=somas
branch_skeleton_data=branch_skeleton_data
combine_close_skeleton_nodes = combine_close_skeleton_nodes
combine_close_skeleton_nodes_threshold=combine_close_skeleton_nodes_threshold

# Setting up the metadata

In [8]:
whole_processing_tiempo = time.time()


"""
Purpose: To process the mesh into a format that can be loaded into the neuron class
and used for higher order processing (how to visualize is included)

"""
if description is None:
    description = "no_description"
if segment_id is None:
    #pick a random segment id
    segment_id = np.random.randint(100000000)
    print(f"picking a random 7 digit segment id: {segment_id}")
    description += "_random_id"


if mesh is None:
    if current_mesh_file is None:
        raise Exception("No mesh or mesh_file file were given")
    else:
        current_neuron = trimesh.load_mesh(current_mesh_file)
else:
    current_neuron = mesh

# ************************ Phase A ********************************

In [11]:
# --- 1) Doing the soma detection
if somas is None:
    soma_mesh_list,run_time,total_soma_list_sdf = sm.extract_soma_center(segment_id,
                                             current_neuron.vertices,
                                             current_neuron.faces)
else:
    soma_mesh_list,run_time,total_soma_list_sdf = somas

# geting the soma centers
if len(soma_mesh_list) <= 0:
    print(f"**** No Somas Found for Mesh {segment_id} so just one mesh")
    soma_mesh_list_centers = []
    if return_no_somas:
        return_value= soma_mesh_list_centers
    raise Exception("Processing of No Somas is not yet implemented yet")
else:
    #compute the soma centers
    print(f"Soma List = {soma_mesh_list}")

    soma_mesh_list_centers = sm.find_soma_centroids(soma_mesh_list)
    print(f"soma_mesh_list_centers = {soma_mesh_list_centers}")

Current Arguments Using (adjusted for decimation):
 large_mesh_threshold= 15000.0 
large_mesh_threshold_inner = 10000.0 
soma_size_threshold = 937.5 
soma_size_threshold_max = 12000.0
outer_decimation_ratio = 0.25
inner_decimation_ratio = 0.25
xvfb-run -n 7377 -s "-screen 0 800x600x24" meshlabserver $@  -i /notebooks/Platinum_Decomp_Fusion/864691135548568516/neuron_864691135548568516.off -o /notebooks/Platinum_Decomp_Fusion/864691135548568516/neuron_864691135548568516_decimated.off -s /notebooks/Platinum_Decomp_Fusion/864691135548568516/decimation_meshlab_25862031.mls
Total found significant pieces before Poisson = [<trimesh.Trimesh(vertices.shape=(191269, 3), faces.shape=(379018, 3))>]
----- working on large mesh #0: <trimesh.Trimesh(vertices.shape=(191269, 3), faces.shape=(379018, 3))>
pre_largest_mesh_path = /notebooks/Platinum_Decomp_Fusion/864691135548568516/neuron_864691135548568516_decimated_largest_piece.off
xvfb-run -n 765 -s "-screen 0 800x600x24" meshlabserver $@  -i /notebo

In [17]:
#--- 2) getting the soma submeshes that are connected to each soma and identifiying those that aren't (and eliminating any mesh pieces inside the soma)

main_mesh_total = current_neuron


#finding the mesh pieces that contain the soma
#splitting the current neuron into distinct pieces
split_meshes = tu.split_significant_pieces(
                            main_mesh_total,
                            significance_threshold=sig_th_initial_split,
                            print_flag=False)

print(f"# total split meshes = {len(split_meshes)}")


#returns the index of the split_meshes index that contains each soma    
containing_mesh_indices = sm.find_soma_centroid_containing_meshes(soma_mesh_list,
                                        split_meshes)

# filtering away any of the inside floating pieces: 
non_soma_touching_meshes = [m for i,m in enumerate(split_meshes)
                 if i not in list(containing_mesh_indices.values())]


#Adding the step that will filter away any pieces that are inside the soma
if len(non_soma_touching_meshes) > 0 and len(soma_mesh_list) > 0:
    """
    *** want to save these pieces that are inside of the soma***
    """

    non_soma_touching_meshes,inside_pieces = sm.filter_away_inside_soma_pieces(soma_mesh_list,non_soma_touching_meshes,
                                    significance_threshold=sig_th_initial_split,
                                    return_inside_pieces = True)                                                      


split_meshes # the meshes of the original mesh
containing_mesh_indices #the mapping of each soma centroid to the correct split mesh
soma_containing_meshes = sm.grouping_containing_mesh_indices(containing_mesh_indices)

soma_touching_meshes = [split_meshes[k] for k in soma_containing_meshes.keys()]


#     print(f"# of non soma touching seperate meshes = {len(non_soma_touching_meshes)}")
#     print(f"# of inside pieces = {len(inside_pieces)}")
print(f"# of soma containing seperate meshes = {len(soma_touching_meshes)}")
print(f"meshes with somas = {soma_containing_meshes}")






# total split meshes = 35
There were 34 pieces found after size threshold
# of soma containing seperate meshes = 1
meshes with somas = {0: [0]}


In [238]:
#--- 3)  Soma Extraction was great (but it wasn't the original soma faces), so now need to get the original soma faces and the original non-soma faces of original pieces

#     sk.graph_skeleton_and_mesh(other_meshes=[soma_meshes])



"""
for each soma touching mesh get the following:
1) original soma meshes
2) significant mesh pieces touching these somas
3) The soma connectivity to each of the significant mesh pieces
-- later will just translate the 


Process: 

1) Final all soma faces (through soma extraction and then soma original faces function)
2) Subtact all soma faces from original mesh
3) Find all significant mesh pieces
4) Backtrack significant mesh pieces to orignal mesh and find connectivity of each to all
   the available somas
Conclusion: Will have connectivity map


"""

soma_touching_mesh_data = dict()

for z,(mesh_idx, soma_idxes) in enumerate(soma_containing_meshes.items()):
    soma_touching_mesh_data[z] = dict()
    print(f"\n\n----Working on soma-containing mesh piece {z}----")

    #1) Final all soma faces (through soma extraction and then soma original faces function)
    current_mesh = split_meshes[mesh_idx]

    current_soma_mesh_list = [soma_mesh_list[k] for k in soma_idxes]

    current_time = time.time()
    mesh_pieces_without_soma = sm.subtract_soma(current_soma_mesh_list,current_mesh,
                                                significance_threshold=250)
    print(f"Total time for Subtract Soam = {time.time() - current_time}")
    current_time = time.time()

    mesh_pieces_without_soma_stacked = tu.combine_meshes(mesh_pieces_without_soma)

    # find the original soma faces of mesh
    soma_faces = tu.original_mesh_faces_map(current_mesh,mesh_pieces_without_soma_stacked,matching=False)
    print(f"Total time for Original_mesh_faces_map for mesh_pieces without soma= {time.time() - current_time}")
    current_time = time.time()
    soma_meshes = current_mesh.submesh([soma_faces],append=True,repair=False)

    # finding the non-soma original faces
    non_soma_faces = tu.original_mesh_faces_map(current_mesh,soma_meshes,matching=False)
    non_soma_stacked_mesh = current_mesh.submesh([non_soma_faces],append=True,repair=False)

    print(f"Total time for Original_mesh_faces_map for somas= {time.time() - current_time}")
    current_time = time.time()

    # 3) Find all significant mesh pieces
    sig_non_soma_pieces,insignificant_limbs = tu.split_significant_pieces(non_soma_stacked_mesh,significance_threshold=limb_threshold,
                                                     return_insignificant_pieces=True)

    print(f"Total time for sig_non_soma_pieces= {time.time() - current_time}")
    current_time = time.time()

    soma_touching_mesh_data[z]["branch_meshes"] = sig_non_soma_pieces

    #4) Backtrack significant mesh pieces to orignal mesh and find connectivity of each to all the available somas
    # get all the seperate mesh faces

    #How to seperate the mesh faces
    seperate_soma_meshes,soma_face_components = tu.split(soma_meshes,only_watertight=False)
    #take the top largest ones depending how many were originally in the soma list
    seperate_soma_meshes = seperate_soma_meshes[:len(soma_mesh_list)]
    soma_face_components = soma_face_components[:len(soma_mesh_list)]

    soma_touching_mesh_data[z]["soma_meshes"] = seperate_soma_meshes

    print(f"Total time for split= {time.time() - current_time}")
    current_time = time.time()



    soma_to_piece_connectivity = dict()
    soma_to_piece_touching_vertices = dict()
    soma_to_piece_touching_vertices_idx = dict()
    limb_root_nodes = dict()
    
    m_vert_graph = tu.mesh_vertex_graph(current_mesh)
    
    for i,curr_soma in enumerate(seperate_soma_meshes):
        (connected_mesh_pieces,
         connected_mesh_pieces_vertices,
         connected_mesh_pieces_vertices_idx) = tu.mesh_pieces_connectivity(
                        main_mesh=current_mesh,
                        central_piece=curr_soma,
                        periphery_pieces = sig_non_soma_pieces,
                        return_vertices = True,
                        return_vertices_idx=True)
        #print(f"soma {i}: connected_mesh_pieces = {connected_mesh_pieces}")
        soma_to_piece_connectivity[i] = connected_mesh_pieces

        soma_to_piece_touching_vertices[i] = dict()
        for piece_index,piece_idx in enumerate(connected_mesh_pieces):
            limb_root_nodes[piece_idx] = connected_mesh_pieces_vertices[piece_index][0]
            
            #find the number of touching groups and save those 
            soma_touching_graph = m_vert_graph.subgraph(connected_mesh_pieces_vertices_idx[piece_index])
            soma_con_comp = [current_mesh.vertices[np.array(list(k)).astype("int")] for k in list(nx.connected_components(soma_touching_graph))]
            soma_to_piece_touching_vertices[i][piece_idx] = soma_con_comp

#         border_debug = False
#         if border_debug:
#             print(f"soma_to_piece_connectivity = {soma_to_piece_connectivity}")
#             print(f"soma_to_piece_touching_vertices = {soma_to_piece_touching_vertices}")


    print(f"Total time for mesh_pieces_connectivity= {time.time() - current_time}")

    soma_touching_mesh_data[z]["soma_to_piece_connectivity"] = soma_to_piece_connectivity

print(f"# of insignificant_limbs = {len(insignificant_limbs)} with trimesh : {insignificant_limbs}")



# Lets have an alert if there was more than one soma disconnected meshes
if len(soma_touching_mesh_data.keys()) > 1:
    raise Exception("More than 1 disconnected meshes that contain somas")

current_mesh_data = soma_touching_mesh_data
soma_containing_idx = 0

#doing inversion of the connectivity and touching vertices
piece_to_soma_touching_vertices = gu.flip_key_orders_for_dict(soma_to_piece_touching_vertices)


# ****Soma Touching mesh Data has the branches and the connectivity (So this is where you end up skipping if you don't have somas)***



----Working on soma-containing mesh piece 0----

inside Soma subtraction
mesh pieces in subtact soma BEFORE the filtering inside pieces = [<trimesh.Trimesh(vertices.shape=(442924, 3), faces.shape=(883241, 3))>, <trimesh.Trimesh(vertices.shape=(108008, 3), faces.shape=(215445, 3))>, <trimesh.Trimesh(vertices.shape=(75957, 3), faces.shape=(151431, 3))>, <trimesh.Trimesh(vertices.shape=(54061, 3), faces.shape=(107833, 3))>, <trimesh.Trimesh(vertices.shape=(42023, 3), faces.shape=(83820, 3))>, <trimesh.Trimesh(vertices.shape=(17933, 3), faces.shape=(35808, 3))>]
There were 6 pieces found after size threshold
mesh pieces in subtact soma AFTER the filtering inside pieces = [<trimesh.Trimesh(vertices.shape=(442924, 3), faces.shape=(883241, 3))>, <trimesh.Trimesh(vertices.shape=(108008, 3), faces.shape=(215445, 3))>, <trimesh.Trimesh(vertices.shape=(75957, 3), faces.shape=(151431, 3))>, <trimesh.Trimesh(vertices.shape=(54061, 3), faces.shape=(107833, 3))>, <trimesh.Trimesh(vertices.shape=(42

# Pause: Investigating the Soma Touching Vertices 

In [27]:
import neuron_visualizations as nviz

In [89]:
nviz.plot_objects(meshes=current_mesh,
scatters=[soma_to_piece_touching_vertices[0][1][0]])

# *** Phase B ***** Skeletonization with MP/MAP Decomp

In [498]:
import preprocessing_vp2 as pre
pre = reload(pre)
sk = reload(sk)
tu = reload(tu)
m_sk = reload(m_sk)
from copy import deepcopy

# Doing the first pass of surface skeletonization

In [499]:
curr_mesh = mesh_data["branch_meshes"][1]

total_border_vertices = dict()
for l_idx,s_dict in piece_to_soma_touching_vertices.items():
    local_b_verts = []
    for sm_idx,sm_bord_verts in s_dict.items():
        local_b_verts.append(np.vstack(sm_bord_verts))
    total_border_vertices[l_idx] = np.vstack(local_b_verts)
    

# for k in soma_to_piece_touching_vertices.keys():
#     if z in soma_to_piece_touching_vertices[k].keys():
#         total_border_vertices[k] = np.vstack(soma_to_piece_touching_vertices[k][z])
        
# possible_soma_border = np.vstack(list(total_border_vertices.values()))

In [500]:
nviz.plot_objects(skeletons=[sk_meshparty_obj.vertices[sk_meshparty_obj.edges]],
                 scatters=[sk_meshparty_obj.vertices[sk_meshparty_obj.root]])

In [501]:
#will eventually get the current root from soma_to_piece_touching_vertices[i]
root_curr = total_border_vertices[1][0]

m_sk = reload(m_sk)
sk_meshparty_obj,limb_mesh_mparty = m_sk.skeletonize_mesh_largest_component(curr_mesh,
                                                        root=root_curr)
m_sk = reload(m_sk)

(segment_branches, #skeleton branches
divided_submeshes, divided_submeshes_idx, #mesh correspondence (mesh and indices)
segment_widths_median) = m_sk.skeleton_obj_to_branches(sk_meshparty_obj,
                                                      mesh = limb_mesh_mparty)

  0%|          | 0/1 [00:00<?, ?it/s]

smooth_neighborhood = 1


100%|██████████| 1/1 [00:01<00:00,  1.66s/it]


/meshAfterParty/meshparty_skeletonize.py:862: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  segment_branches = np.array([sk_meshparty_obj.vertices[np.vstack([k[:-1],k[1:]]).T] for k in segments])
/meshAfterParty/meshparty_skeletonize.py:895: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  segment_branches = new_segment_branches


branches_touching_root = [27]
empty_indices % = 0.0
 conflict_indices % = 0.004901482977093922



AFTER face_lookup_resolved_test


/meshAfterParty/trimesh_utils.py:735: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  total_submeshes_idx =np.array(list(total_submeshes_idx.values()))


In [502]:
#using the width threshold
width_threshold = 450
size_threshold = 1000

#gettin the branches that should be passed through MAP skeletonization
pieces_above_threshold = np.where(segment_widths_median>width_threshold)[0]

#getting the correspondnece info for those MAP qualifying
width_large = segment_widths_median[pieces_above_threshold]
sk_large = [segment_branches[k] for k in pieces_above_threshold]
mesh_large_idx = [divided_submeshes_idx[k] for k in pieces_above_threshold]

if len(mesh_large_idx) > 0: #will only continue processing if found MAP candidates
    #finds the connectivity edges of all the MAP candidates
    mesh_large_connectivity = tu.mesh_list_connectivity(meshes = mesh_large_idx,
                            main_mesh = limb_mesh_mparty,
                            print_flag = False)
    """
    --------------- Grouping MAP candidates ----------------
    Purpose: Will see what mesh pieces should be grouped together
    to pass through CGAL skeletonization
    
    
    Pseudocode: 
    1) build a networkx graph with all nodes for mesh_large_idx indexes
    2) Add the edges
    3) Find the connected components
    4) Find sizes of connected components
    5) For all those connected components that are of a large enough size, 
    add the mesh branches and skeletons to the final list


    """
    G = nx.Graph()
    G.add_nodes_from(np.arange(len(mesh_large_idx)))
    G.add_edges_from(mesh_large_connectivity)
    conn_comp = list(nx.connected_components(G))

    filtered_pieces = []

    sk_large_size_filt = []
    mesh_large_idx_size_filt = []
    width_large_size_filt = []

    for cc in conn_comp:
        total_cc_size = np.sum([len(mesh_large_idx[k]) for k in cc])
        if total_cc_size>size_threshold:
            #print(f"cc ({cc}) passed the size threshold because size was {total_cc_size}")
            filtered_pieces.append(pieces_above_threshold[list(cc)])
            
    #filtered_pieces: will have the indexes of all the branch candidates that should  be 
    #grouped together and passed through MAP skeletonization

    if len(filtered_pieces) > 0:
        #all the pieces that will require MAP mesh correspondence and skeletonization
        #(already organized into their components)
        mesh_pieces_for_MAP = [limb_mesh_mparty.submesh([np.concatenate(divided_submeshes_idx[k])],append=True,repair=False) for k in filtered_pieces]

        pieces_idx_MP = np.setdiff1d(np.arange(len(divided_submeshes_idx)),np.concatenate(filtered_pieces))
        
        """
        Old Way: Finding connectivity of pieces through
        mesh_idx_MP = [divided_submeshes_idx[k] for k in pieces_idx_MP]

        mesh_large_connectivity_MP = tu.mesh_list_connectivity(meshes = mesh_idx_MP,
                                main_mesh = limb_mesh_mparty,
                                print_flag = False)
                                
        New Way: going to use skeleton connectivity to determine
        connectivity of pieces
        
        Pseudocode: 
        1)
        
        """
        
        skeleton_MP = [segment_branches[k] for k in pieces_idx_MP]
        skeleton_connectivity_MP = sk.skeleton_list_connectivity(
                                        skeletons=skeleton_MP
                                        )
        
        G = nx.Graph()
        G.add_nodes_from(np.arange(len(mesh_idx_MP)))
        G.add_edges_from(skeleton_connectivity_MP)
        sublimbs_MP = list(nx.connected_components(G))
        sublimbs_MP_orig_idx = [pieces_idx_MP[list(k)] for k in sublimbs_MP]


        #concatenate into sublimbs the skeletons and meshes
        sublimb_mesh_idx_branches_MP = [divided_submeshes_idx[k] for k in sublimbs_MP_orig_idx]
        sublimb_mesh_branches_MP = [[limb_mesh_mparty.submesh([ki],append=True,repair=False)
                                    for ki in k] for k in sublimb_mesh_idx_branches_MP]
        sublimb_meshes_MP = [limb_mesh_mparty.submesh([np.concatenate(k)],append=True,repair=False)
                                                     for k in sublimb_mesh_idx_branches_MP]
        sublimb_skeleton_branches = [segment_branches[k] for k in sublimbs_MP_orig_idx]
        widths_MP = [segment_widths_median[k] for k in sublimbs_MP_orig_idx]

else: #if no pieces were determine to need MAP processing
    print("No MAP processing needed: just returning the Meshparty skeletonization and mesh correspondence")
    raise Exception("Returning MP correspondence")


#         for indiv_cc in cc:
#             sk_large_size_filt.append(sk_large[indiv_cc])
#             mesh_large_idx_size_filt.append(mesh_large_idx[indiv_cc])
#             width_large_size_filt.append(width_large[indiv_cc])

# nviz.plot_objects(main_mesh=tu.combine_meshes([limb_mesh_mparty,current_neuron["S0"].mesh]),
#                   main_mesh_color="green",
#     skeletons=sk_large_size_filt,
#      meshes=[limb_mesh_mparty.submesh([k],append=True) for k in mesh_large_idx_size_filt],
#       meshes_colors="red")

In [503]:
mesh_pieces_for_MAP #trimesh pieces that should go through CGAL skeletonization
sublimb_meshes_MP #trimesh pieces that have already been passed through MP skeletonization (may not need)

# -- the decomposition information ---
sublimb_mesh_branches_MP #the mesh branches for all the disconnected sublimbs
sublimb_mesh_idx_branches_MP #The mesh branches idx that have already passed through MP skeletonization
sublimb_skeleton_branches #the skeleton bnraches for all the sublimbs
widths_MP #the mesh branches widths for all the disconnected groups

[array([149.8654772 , 418.55880666, 311.43380537]),
 array([405.8431516]),
 array([414.99335148]),
 array([400.39759195]),
 array([402.46626928, 186.91510311]),
 array([421.16696781, 127.10122558, 328.15344523])]

# Finding which of the sublimbs the soma touching vertices belogs to

In [504]:
return_value = pre.filter_soma_touching_vertices_dict_by_mesh(
    mesh = mesh_pieces_for_MAP[0],
    curr_piece_to_soma_touching_vertices = piece_to_soma_touching_vertices[1]
    )
return_value

{0: [TrackedArray([[580478.6, 771669. , 885399.6],
                [579293.8, 772063.6, 885518. ],
                [580218.1, 771431.4, 885658.1],
                [579967.5, 772038.8, 885108. ],
                [579810. , 771918. , 885108. ],
                [579749.8, 771992.9, 885150.2],
                [580156.6, 771634.1, 885833.4],
                [579800.9, 771569.4, 885956.9],
                [579190.2, 771672.2, 885807.2],
                [580298.4, 771508.4, 885558.6],
                [579088.1, 771776.2, 885706. ],
                [580237.9, 771731.7, 885095.4],
                [580083. , 771839.2, 885097.5],
                [579342.4, 771610.4, 885914.6],
                [580324.5, 771687. , 885606.8],
                [579682.6, 772060.9, 885191. ],
                [579552.9, 771584.2, 885938.3],
                [579148.5, 771750. , 885801. ],
                [580399.5, 771822.4, 885275.6],
                [580209.7, 771929.2, 885127.6],
                [579396.1, 771929. , 

# Testing out the Soma Extension and the Changing of Mesh Correspondence 

In [505]:
whole_sk_MP = sk.stack_skeletons(segment_branches)

In [506]:
return_info = sk.create_soma_extending_branches(current_skeleton=whole_sk_MP,
                              skeleton_mesh=branch,
                              soma_to_piece_touching_vertices=piece_to_soma_touching_vertices[1],
                              return_endpoints_must_keep=True,
                                 return_created_branch_info=True)
new_sk,endpts,new_branch_info = return_info

Adding new branch to skeleton
Adding new branch to skeleton


In [507]:
#finding the branches where the new segment was added

"""
Pseudocode:
1) Get the newly added branch (and the original vertex which is the first row)
2) Find the branches that have that coordinate (could be multiple)
2b) If multiple branches: Pick a winning branch that has the most of the
    soma touching vertices
3) Find the mesh and skeleton of the winning branch
4) Add newly created branch to skeleton and divide the skeleton into branches (could make 2 or 3)
5) Run Adaptive mesh correspondnece using branches and mesh
6) Remove the original branch and mesh correspondence and replace with the multiples

"""
#maps the 


#for sm_ind in new_branch_info
# sm_idx = 0
# b_vert_idx = 0
verbose = True

for sm_idx in new_branch_info.keys():
    for b_vert_idx in new_branch_info[sm_idx].keys():

        #1) Get the newly added branch (and the original vertex which is the first row)
        br_new,sm_bord_verts = new_branch_info[sm_idx][b_vert_idx].values()

        curr_soma_to_piece_touching_vertices_MP = {sm_idx:[sm_bord_verts]}
        endpoints_must_keep_MP = {sm_idx:[br_new[0][1]]}

        orig_vertex = br_new[0][0]
        
        
        #2) Find the branches that have that coordinate (could be multiple)
        match_sk_branches = sk.find_branch_skeleton_with_specific_coordinate(segment_branches,
            current_coordinate=orig_vertex)

        #2b) If multiple branches: Pick a winning branch that has the most of the
        #    soma touching vertices
        if len(match_sk_branches) > 1:
            bord_verts_tree = KDTree(sm_bord_verts.reshape(-1,3))
            winning_branch = match_sk_branches[0]
            dist,_ = bord_verts_tree.query(divided_submeshes[winning_branch].vertices)
            winning_branch_n_bord_verts = np.sum(dist == 0)

            for i in range(1,len(match_sk_branches)):
                curr_branch = match_sk_branches[i]
                dist,_ = bord_verts_tree.query(divided_submeshes[winning_branch].vertices)
                n_bord_verts = np.sum(dist == 0)
                if n_bord_verts>winning_branch_n_bord_verts:
                    winning_branch_n_bord_verts = n_bord_verts
                    winning_branch = curr_branch
        elif len(match_sk_branches) == 1:
            winning_branch = match_sk_branches[0]
        else:
            raise Exception("No matching branches found for soma extending point")

        if verbose:
            print(f"Winning Branch = {winning_branch}")

        #3) Find the mesh and skeleton of the winning branch
        winning_branch_mesh = divided_submeshes[winning_branch]
        winning_branch_mesh_idx = divided_submeshes_idx[winning_branch]
        winning_branch_sk = segment_branches[winning_branch]

        #4) Add newly created branch to skeleton and divide the skeleton into branches (could make 2 or 3)
        extended_skeleton_to_soma = sk.stack_skeletons([winning_branch_sk,br_new])
        soma_extending_branches = sk.decompose_skeleton_to_branches(extended_skeleton_to_soma)

        #5) Run Adaptive mesh correspondnece using branches and mesh
        local_correspondnece_MP = pre.mesh_correspondence_first_pass(mesh=winning_branch_mesh,
                                      skeleton=extended_skeleton_to_soma)

        local_correspondence_revised = pre.correspondence_1_to_1(mesh=winning_branch_mesh,
                                                    local_correspondence=local_correspondnece_MP,
                                                    curr_limb_endpoints_must_keep=endpoints_must_keep_MP,
                                                    curr_soma_to_piece_touching_vertices=curr_soma_to_piece_touching_vertices_MP)

        # All the things that should be revised:
    #     segment_branches, #skeleton branches
    #     divided_submeshes, divided_submeshes_idx, #mesh correspondence (mesh and indices)
    #     segment_widths_median


        new_submeshes = [k["branch_mesh"] for k in local_correspondence_revised.values()]
        new_submeshes_idx = [winning_branch_mesh_idx[k["branch_face_idx"]] for k in local_correspondence_revised.values()]
        new_skeletal_branches = [k["branch_skeleton"] for k in local_correspondence_revised.values()]

        #calculate the new width
        ray_inter = tu.ray_pyembree.RayMeshIntersector(limb_mesh_mparty)
        new_widths = []
        for new_s_idx in new_submeshes_idx:
            curr_ray_distance = tu.ray_trace_distance(mesh=limb_mesh_mparty, 
                                face_inds=new_s_idx,
                               ray_inter=ray_inter)
            new_widths.append(np.median(curr_ray_distance[curr_ray_distance!=0]))


        #6) Remove the original branch and mesh correspondence and replace with the multiples
    #     new_segment_branches = deepcopy(segment_branches)
    #     new_divided_submeshes = deepcopy(divided_submeshes)
    #     new_divided_submeshes_idx = deepcopy(divided_submeshes_idx)
    #     new_segment_widths_median = deepcopy(segment_widths_median)

        segment_branches = np.delete(segment_branches,winning_branch)
        segment_branches = np.append(segment_branches,new_skeletal_branches,axis=0)

        divided_submeshes = np.delete(divided_submeshes,winning_branch)
        divided_submeshes = np.append(divided_submeshes,new_submeshes,axis=0)

        divided_submeshes_idx = np.delete(divided_submeshes_idx,winning_branch)
        divided_submeshes_idx = np.append(divided_submeshes_idx,new_submeshes_idx,axis=0)

        segment_widths_median = np.delete(segment_widths_median,winning_branch)
        segment_widths_median = np.append(segment_widths_median,new_widths,axis=0)
    



Winning Branch = 27





--- Working on 1-to-1 correspondence-----
max(original_labels),len(original_labels) = (2, 3)
empty_indices % = 0.12254335260115606
 conflict_indices % = 0.0403339755940912



AFTER face_lookup_resolved_test
Took 0 iterations to expand the label back



Winning Branch = 5





--- Working on 1-to-1 correspondence-----
max(original_labels),len(original_labels) = (2, 3)
empty_indices % = 0.02508774210321071
 conflict_indices % = 0.02335456475583864



AFTER face_lookup_resolved_test
Took 0 iterations to expand the label back


In [508]:
# # All the things that should be revised:
# segment_branches, #skeleton branches
# divided_submeshes, divided_submeshes_idx, #mesh correspondence (mesh and indices)
# segment_widths_median


# new_submeshes = [k["branch_mesh"] for k in local_correspondence_revised.values()]
# new_submeshes_idx = [winning_branch_mesh_idx[k["branch_face_idx"]] for k in local_correspondence_revised.values()]
# new_skeletal_branches = [k["branch_skeleton"] for k in local_correspondence_revised.values()]

# #calculate the new width
# ray_inter = tu.ray_pyembree.RayMeshIntersector(limb_mesh_mparty)
# new_widths = []
# for new_s_idx in new_submeshes_idx:
#     curr_ray_distance = tu.ray_trace_distance(mesh=limb_mesh_mparty, 
#                         face_inds=new_s_idx,
#                        ray_inter=ray_inter)
#     new_widths.append(np.median(curr_ray_distance[curr_ray_distance!=0]))


# #Reassigning Now
# new_segment_branches = deepcopy(segment_branches)
# new_divided_submeshes = deepcopy(divided_submeshes)
# new_divided_submeshes_idx = deepcopy(divided_submeshes_idx)
# new_segment_widths_median = deepcopy(segment_widths_median)

# new_segment_branches = np.delete(new_segment_branches,winning_branch)
# new_segment_branches = np.append(new_segment_branches,new_skeletal_branches,axis=0)

# new_divided_submeshes = np.delete(new_divided_submeshes,winning_branch)
# new_divided_submeshes = np.append(new_divided_submeshes,new_submeshes,axis=0)

# new_divided_submeshes_idx = np.delete(new_divided_submeshes_idx,winning_branch)
# new_divided_submeshes_idx = np.append(new_divided_submeshes_idx,new_submeshes_idx,axis=0)

# new_segment_widths_median = np.delete(new_segment_widths_median,winning_branch)
# new_segment_widths_median = np.append(new_segment_widths_median,new_widths,axis=0)


In [509]:
nviz.plot_objects(meshes=divided_submeshes,
                  meshes_colors="random",
                 skeletons=segment_branches,
                  skeletons_colors="random"
                 )

In [452]:
nviz.plot_objects(meshes=[k["branch_mesh"] for k in local_correspondence_revised.values()],
                  meshes_colors="random",
                skeletons=[k["branch_skeleton"] for k in local_correspondence_revised.values()],
                  skeletons_colors="random",
                 scatters=[orig_vertex],
                 scatter_size=1)

# Doing the CGAL skeletonization

In [134]:

global_start_time = time.time()
endpoints_must_keep = dict()

perform_cleaning_checks = False


#For the limb correspondence
limb_correspondence = dict()
soma_containing_idx= 0

if branch_skeleton_data is None:
    for j,(soma_containing_mesh_idx,mesh_data) in enumerate(soma_touching_mesh_data.items()):

#                 sk_debug = True
#                 if sk_debug:
#                     import system_utils as su
#                     su.compressed_pickle(mesh_data["branch_meshes"],
#                                         "ordered_branch_meshes")
#                 raise Exception("Done exporting branches")

        
        
        print(f"\n-- Working on Soma Continaing Mesh {j}--")
        current_branches = mesh_data["branch_meshes"]

        #skeletonize each of the branches
        total_skeletons = []

        for z,branch in enumerate(current_branches):
            print(f"\n    -- Working on branch {z}--")
            if z != 1:
                continue
            mesh_start_time = time.time()
                
            # ---- 0) Generating the Clean skeletons  -------------------------------------------#
            total_border_vertices = dict()
            for k in soma_to_piece_touching_vertices.keys():
                if z in soma_to_piece_touching_vertices[k].keys():
                    total_border_vertices[k] = np.vstack(soma_to_piece_touching_vertices[k][z])
            
            
            
            #build the soma to piece touching vertices dictionary for this neuorn
            curr_soma_to_piece_touching_vertices = dict()
            for s_index,v in soma_to_piece_touching_vertices.items():
                if z not in v:
                    continue
                curr_soma_to_piece_touching_vertices[s_index] = soma_to_piece_touching_vertices[s_index][z]
                
            cleaned_branch,curr_limb_endpoints_must_keep = sk.skeletonize_and_clean_connected_branch_CGAL(
                mesh=branch,
                curr_soma_to_piece_touching_vertices=curr_soma_to_piece_touching_vertices,
                total_border_vertices=total_border_vertices,
                filter_end_node_length=filter_end_node_length,
                perform_cleaning_checks=perform_cleaning_checks,
                combine_close_skeleton_nodes = combine_close_skeleton_nodes,
                combine_close_skeleton_nodes_threshold=combine_close_skeleton_nodes_threshold)
            
            
            endpoints_must_keep[z] = curr_limb_endpoints_must_keep
            
            #do the cleanin ghtat removes loops from branches
            print(f"After skeletonization and cleaning skleelton size = {cleaned_branch.shape}")
            total_skeletons.append(cleaned_branch)


            print(f"    Total time for cleaning of branch {z}: {time.time() - clean_time}")
            if len(cleaned_branch) == 0:
                raise Exception(f"Found a zero length skeleton for limb {z} of trmesh {branch}")
            total_skeletons.append(cleaned_branch)
            
            
            
            # ---- 1) Generating Initial Mesh Correspondence -------------------------------------------#
            start_time = time.time()
            limb_idx = z
            
            print(f"Working on limb correspondence for #{limb_idx}")
            local_correspondence = pre.mesh_correspondence_first_pass(mesh=branch,
                                                                 skeleton=cleaned_branch,
                                                                 distance_by_mesh_center=distance_by_mesh_center)
            limb_correspondence[limb_idx] = local_correspondence
            
            
            print(f"Total time for decomposition = {time.time() - start_time}")
            
            #------------- 2) Doing Some checks on the initial corespondence -------- #
            
            perform_cleaning_checks = True       
            if perform_cleaning_checks:
                pre.check_skeletonization_and_decomp(skeleton=cleaned_branch,
                                                local_correspondence=local_correspondence)
                
            # -------3) Finishing off the face correspondence so get 1-to-1 correspondence of mesh face to skeletal piece
            local_correspondence_revised = pre.correspondence_1_to_1(mesh=branch,
                                            local_correspondence=local_correspondence,
                                            curr_limb_endpoints_must_keep=curr_limb_endpoints_must_keep,
                                            curr_soma_to_piece_touching_vertices=curr_soma_to_piece_touching_vertices)

            print(f"Total time for limb mesh processing = {time.time() - mesh_start_time}")

        soma_touching_mesh_data[j]["branch_skeletons_cleaned"] = total_skeletons

    print(f"Total time for skeletonization and mesh corespondence = {time.time() - global_start_time}")

else:
    print("****Skipping skeleton cleaning and USING THE PRE-COMPUTED SKELETONS ****")
    soma_touching_mesh_data[0]["branch_skeletons_cleaned"] =branch_skeleton_data
    
current_mesh_data = soma_touching_mesh_data


-- Working on Soma Continaing Mesh 0--

    -- Working on branch 0--

    -- Working on branch 1--
     Starting Screened Poisson
xvfb-run -n 4715 -s "-screen 0 800x600x24" meshlabserver $@  -i /notebooks/Platinum_Decomp_Fusion/temp/None.off -o /notebooks/Platinum_Decomp_Fusion/temp/None_poisson.off -s /notebooks/Platinum_Decomp_Fusion/temp/poisson_922211.mls
removed temporary input file: /notebooks/Platinum_Decomp_Fusion/temp/None.off
removed temporary output file: /notebooks/Platinum_Decomp_Fusion/temp/None_poisson.off
Using the close holes feature
xvfb-run -n 6153 -s "-screen 0 800x600x24" meshlabserver $@  -i /notebooks/Platinum_Decomp_Fusion/temp/neuron_14072.off -o /notebooks/Platinum_Decomp_Fusion/temp/neuron_14072_fill_holes.off -s /notebooks/Platinum_Decomp_Fusion/temp/fill_holes_360365.mls
removed temporary input file: /notebooks/Platinum_Decomp_Fusion/temp/neuron_14072.off
removed temporary output file: /notebooks/Platinum_Decomp_Fusion/temp/neuron_14072_fill_holes.off
----


Total Mesh subtraction time = 43.4393



len_subgraphs AT BEGINNING of the loop


all graph is one component!

Total time for skeleton stitching = 0.38226771354675293
The process was using a temp folder
    Total time for skeletonizing branch: 87.33870434761047
skipping soma 0 because closest skeleton node was already end node
Adding new branch to skeleton
    Total time for Fixing Skeleton Soma Endpoint Extension : 15.409197807312012
filter_end_node_length = 4001
Using Distance measure skeletal_distance
Going to ignore certain endnodes that are 5000 nm close to soma border vertices
Number of end_nodes BEFORE filtering = 120
Using an already specified end node: 1673 with index 59checking was correct node end_nodes[index] = 1673
Using an already specified end node: 1571 with index 55checking was correct node end_nodes[index] = 1571
May Eliminate end_node 59: 1673 because path_len to soma border was 2070.282670892157
single_node_to_eliminate = 59
all_single_nodes_to_eliminate = [59, 55, 59]



********COMBINING CLOSE SKELETON NODES WITHIN 700 DISTANCE**********
Found 0 valid paths to replace
valid_paths = []
valid_path_lengths = []
No valid paths found so just returning the original
After skeletonization and cleaning skleelton size = (3308, 2, 3)
    Total time for cleaning of branch 1: 7000.877564907074
Working on limb correspondence for #1



Total time for decomposition = 44.051918745040894

    -- Working on branch 2--

    -- Working on branch 3--

    -- Working on branch 4--

    -- Working on branch 5--
Total time for skeletonization = 154.63969540596008


In [438]:
curr_soma_to_piece_touching_vertices

{0: [TrackedArray([[580478.6, 771669. , 885399.6],
                [579293.8, 772063.6, 885518. ],
                [580218.1, 771431.4, 885658.1],
                [579967.5, 772038.8, 885108. ],
                [579810. , 771918. , 885108. ],
                [579749.8, 771992.9, 885150.2],
                [580156.6, 771634.1, 885833.4],
                [579800.9, 771569.4, 885956.9],
                [579190.2, 771672.2, 885807.2],
                [580298.4, 771508.4, 885558.6],
                [579088.1, 771776.2, 885706. ],
                [580237.9, 771731.7, 885095.4],
                [580083. , 771839.2, 885097.5],
                [579342.4, 771610.4, 885914.6],
                [580324.5, 771687. , 885606.8],
                [579682.6, 772060.9, 885191. ],
                [579552.9, 771584.2, 885938.3],
                [579148.5, 771750. , 885801. ],
                [580399.5, 771822.4, 885275.6],
                [580209.7, 771929.2, 885127.6],
                [579396.1, 771929. , 

In [437]:
curr_limb_endpoints_must_keep

{0: array([[579716.        , 769970.        , 884968.        ],
        [574112.13372093, 784665.63837209, 890838.61395349]])}

In [161]:
# curr_branch = local_correspondence[8]
# nviz.plot_objects(meshes=curr_branch["correspondence_mesh"],
#                 skeletons=[curr_branch["branch_skeleton"]])